In [1]:
%%capture
!pip install pygame
!pip install pygad

!git clone https://github.com/karinemiras/evoman_framework.git tmp
!cp -r /kaggle/working/tmp/* /kaggle/working/
!rm -R /kaggle/working/tmp

In [2]:
import sys, os
import numpy as np
import pandas as pd

import random

from evoman.environment import Environment
from evoman.controller import Controller
from demo_controller import player_controller
from deap import base, creator, tools, algorithms
import multiprocessing
import optuna

import seaborn as sns
import matplotlib.pyplot as plt

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
def parameter_count(hidden_neurons):
    if hidden_neurons>0:
        n_w = (20*hidden_neurons) + (hidden_neurons*5)
        n_b = hidden_neurons + 5
        return n_w + n_b
    else:
        return (20*5)+5
    
def fitness_function(solution):
    sum_gain = 0
    for enemy in range(1,9):
        env.update_parameter("enemies",[enemy])
        res = env.play(np.array(solution))
        gain = res[1]-res[2]
        sum_gain += gain
    return (sum_gain,)

"""
def fitness_function(solution):
    defeated = 0
    for enemy in range(1,9):
        env.update_parameter("enemies",[enemy])
        res = env.play(np.array(solution))
        if res[2]<=0:
            defeated += 1
    return (defeated,)
"""

'\ndef fitness_function(solution):\n    defeated = 0\n    for enemy in range(1,9):\n        env.update_parameter("enemies",[enemy])\n        res = env.play(np.array(solution))\n        if res[2]<=0:\n            defeated += 1\n    return (defeated,)\n'

In [4]:
def cxWholeAritmetic(parent1,parent2,alpha):
    child1 = (alpha*np.array(parent1))+((1-alpha)*np.array(parent2))
    child2 = (alpha*np.array(parent2))+((1-alpha)*np.array(parent1))
    return creator.Individual(child1),creator.Individual(child2)

def cxBlend(parent1,parent2,alpha):
    gamma = (1+2*alpha)*np.random.uniform(size=(len(parent1,))) - alpha
    child1 = (gamma*np.array(parent1))+((1-gamma)*np.array(parent2))
    child2 = (gamma*np.array(parent2))+((1-gamma)*np.array(parent1))
    return creator.Individual(child1),creator.Individual(child2)

In [5]:
def mutGaussian(ind,sigma,indpb):
    mutation = [np.random.normal(0,sigma) if np.random.uniform()<indpb else 0 for i in range(len(ind))]
    return creator.Individual(np.array(ind) + np.array(mutation))

def mutGaussianAdaptive(ind,sigma,tau,indpb):
    new_sigma = sigma * np.exp(np.random.normal(0,tau,size=(len(ind),)))
    new_sigma[new_sigma<0.1] = 0.1
    mutation = [np.random.normal(0,s) if np.random.uniform()<indpb else 0 for s in new_sigma]
    return creator.Individual(np.array(ind) + np.array(mutation))

In [6]:
def best_initializer(pop_size,in_sigma,in_mutpb,in_tau):
    pop = np.random.choice(final_results['solution'],pop_size)
    pop = [creator.Individual(ind) for ind in pop]
    for i in range(len(pop)):
        if np.random.uniform() < in_mutpb:
            pop[i] = mutGaussianAdaptive(pop[i],in_sigma,in_tau,1)
    return pop

In [7]:
def genetic_algorithm(in_sigma,in_tau,in_mutpb,pop_size,n_offspring,tournsize_parent,cxpb,cx_type,
                      alpha,mutpb,mut_type,sigma,indpb,tau,tournsize_survival,ngen,verbose):
    
    global individual_size,env,creator,stats
    
    toolbox = base.Toolbox()
    toolbox.register("weight_bin", np.random.uniform,-0,0)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.weight_bin, n=individual_size)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", fitness_function)

    pop = best_initializer(pop_size,in_sigma,in_mutpb,in_tau)
    hof = {'best_solution':-np.inf,'max_value':-np.inf}

    with multiprocessing.Pool() as pool:
        # evaluate initial population
        for ind, fit in zip(pop, pool.map(toolbox.evaluate,pop)):
            ind.fitness.values = fit

        # record best solution
        for ind in pop:
            if ind.fitness.values[0]>hof['max_value']:
                hof['best_solution'] = ind
                hof['max_value'] = ind.fitness.values[0]

        logbook = tools.Logbook()
        logbook.header = ['gen', 'nevals'] + (stats.fields if stats else []) + ['max_hof']
        record = stats.compile(pop) if stats else {}

        logbook.record(gen=0, nevals=len(pop), **record, max_hof=hof['max_value'])
        if verbose==1:
            print(logbook.stream)

        for g in range(ngen):
            # select parents
            parents = tools.selTournament(individuals=pop,k=n_offspring*pop_size,tournsize=tournsize_parent)

            # create offspring
            offspring = []
            for child1, child2 in zip(parents[::2], parents[1::2]):
                if np.random.uniform() < cxpb:
                    if cx_type=="whole_aritmetic":
                        if alpha=="random":
                            alpha=np.random.uniform()
                            child1,child2 = cxWholeAritmetic(child1, child2, alpha)
                        else:
                            child1,child2 = cxWholeAritmetic(child1, child2, alpha)
                    elif cx_type=="blend":
                        if alpha=="random":
                            alpha=np.random.uniform()
                            child1,child2 = cxBlend(child1, child2, alpha)
                        else:
                            child1,child2 = cxBlend(child1, child2, alpha)
                offspring.append(child1)
                offspring.append(child2)

            # apply mutation to offspring
            for i in range(len(offspring)):
                if np.random.uniform() < mutpb:
                    if mut_type=="normal":
                        if indpb=="random":
                            indpb=np.random.uniform()
                            offspring[i] = mutGaussian(offspring[i],sigma,indpb)
                        else:
                            offspring[i] = mutGaussian(offspring[i],sigma,indpb)
                    elif mut_type=="adaptive":
                        if indpb=="random":
                            indpb=np.random.uniform()
                            offspring[i] = mutGaussianAdaptive(offspring[i],sigma,indpb,tau)
                        else:
                            offspring[i] = mutGaussianAdaptive(offspring[i],sigma,indpb,tau)       

            # evaluate offspring
            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            for ind, fit in zip(invalid_ind, pool.map(toolbox.evaluate,invalid_ind)):
                    ind.fitness.values = fit

            # record best solution
            for ind in offspring:
                if ind.fitness.values[0]>hof['max_value']:
                    hof['best_solution'] = ind
                    hof['max_value'] = ind.fitness.values[0]

            # select the next generation of the population
            pop = tools.selTournament(individuals=offspring,k=pop_size,tournsize=tournsize_survival)

            record = stats.compile(pop) if stats else {}
            logbook.record(gen=g+1, nevals=len(invalid_ind), **record, max_hof=hof['max_value'])
            if verbose==1:
                print(logbook.stream)
        pool.close()
        pool.join()

    return hof,logbook

In [8]:
hidden_neurons = 10
individual_size = parameter_count(hidden_neurons)

headless = True
if headless:
    os.environ["SDL_VIDEODRIVER"] = "dummy"

env = Environment(playermode="ai",
                  player_controller=player_controller(hidden_neurons),
                  speed="fastest",
                  enemymode="static",
                  level=2,
                  visuals=False,
                  logs="off")

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("Mean", np.mean)
stats.register("Max", np.max)
stats.register("Std", np.std)

ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5704:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM default


In [9]:
def objective(trial):
    
    params = {'in_sigma':trial.suggest_float("in_sigma", 0.1, 1., step=0.1),
              'in_tau':trial.suggest_float("in_tau", 0.1, 1., step=0.1),
              'in_mutpb':trial.suggest_float("in_mutpb", 0.1, 1., step=0.1),
              'pop_size':trial.suggest_int("pop_size", 50,50,step=50),
              'n_offspring':trial.suggest_int("n_offspring", 1,5,step=1),
              'tournsize_parent':trial.suggest_int("tournsize_parent", 1,10,step=1),
              'cxpb':trial.suggest_float("cxpb", 0.1, 1., step=0.1),
              'cx_type':trial.suggest_categorical("cx_type",["whole_aritmetic","blend"]),
              'alpha':trial.suggest_categorical("alpha",np.arange(0.1,1.1,0.1).tolist()+["random"]),
              'mutpb':trial.suggest_float("mutpb", 0.1, 1., step=0.1),
              'mut_type':trial.suggest_categorical("mut_type",["normal","adaptive"]),
              'sigma':trial.suggest_float("sigma", 0.5, 3, step=0.5),
              'indpb':trial.suggest_categorical("indpb",np.arange(0.1,1.1,0.1).tolist()+["random"]),
              'tau':trial.suggest_float("tau", 0.1, 1., step=0.1),
              'tournsize_survival':trial.suggest_int("tournsize_survival", 1,10,step=1),
              'ngen':trial.suggest_int("ngen", 30,30,step=20),
              'verbose':trial.suggest_int("verbose", 1,1,step=1)}
    
    global hof_tuning
    iteration_logs = []
    for i in range(1):
        hof,logs = genetic_algorithm(**params)
        logs_pd = pd.DataFrame(logs)
        iteration_logs.append(logs_pd)
        hof_tuning.append(hof)
    
    return pd.concat(iteration_logs)['max_hof'].max()


In [10]:
import pickle
with open('/kaggle/input/tune-res-generalist/final_solutions_v2.pickle', 'rb') as handle:
    final_results = pickle.load(handle)

In [ ]:
hof_tuning = []

study = optuna.create_study(study_name="GA", direction="maximize")

study.optimize(objective, n_trials=500)

[I 2023-10-15 19:31:03,816] A new study created in memory with name: GA


gen	nevals	Mean   	Max  	Std    	max_hof
0  	50    	116.208	356.8	207.534	356.8  
1  	48    	93.1   	356.8	199.397	356.8  
2  	49    	-14.964	356.8	182.08 	356.8  
3  	49    	113.904	356.8	186.968	356.8  
4  	49    	138.232	356.8	212.392	356.8  
5  	50    	89.164 	356.8	155.353	356.8  
6  	50    	139.488	356.8	195.822	356.8  
7  	49    	38.68  	356.8	186.247	356.8  
8  	49    	146.88 	356.8	219.01 	356.8  
9  	49    	55.012 	356.8	138.956	356.8  
10 	50    	61.644 	356.8	135.333	356.8  
11 	49    	60.044 	356.8	127.05 	356.8  
12 	50    	-21.432	70.4 	88.4992	356.8  
13 	48    	-4.6   	85   	76.559 	356.8  
14 	44    	62.896 	124  	63.7151	356.8  
15 	48    	14.72  	124  	74.221 	356.8  
16 	48    	80.94  	164  	76.8118	356.8  
17 	48    	15.34  	164  	103.53 	356.8  
18 	49    	35.88  	242  	90.8624	356.8  
19 	49    	43.996 	242  	103.905	356.8  
20 	50    	44.56  	164  	87.0056	356.8  
21 	50    	65.02  	164  	90.9096	356.8  
22 	47    	43.032 	164  	120.873	356.8  
23 	49    	25.94

[I 2023-10-15 19:35:17,498] Trial 0 finished with value: 356.8000000000009 and parameters: {'in_sigma': 0.9, 'in_tau': 0.4, 'in_mutpb': 0.2, 'pop_size': 50, 'n_offspring': 1, 'tournsize_parent': 8, 'cxpb': 0.7000000000000001, 'cx_type': 'whole_aritmetic', 'alpha': 0.7000000000000001, 'mutpb': 0.9, 'mut_type': 'adaptive', 'sigma': 2.5, 'indpb': 0.6, 'tau': 0.5, 'tournsize_survival': 5, 'ngen': 30, 'verbose': 1}. Best is trial 0 with value: 356.8000000000009.


30 	46    	135.736	174.2	86.4945	356.8  
gen	nevals	Mean    	Max  	Std    	max_hof
0  	50    	-246.632	197.2	194.675	197.2  
1  	88    	1.516   	197.2	127.983	197.2  
2  	93    	154.856 	261  	106.549	261    
3  	97    	246.656 	261  	31.4359	261    
4  	91    	261     	261  	5.68434e-14	261    
5  	94    	261.696 	276  	2.95736    	276    
6  	92    	265.44  	276  	26.8035    	276    
7  	97    	276     	276  	5.68434e-14	276    
8  	97    	275.584 	276  	2.912      	276    
9  	94    	271.248 	293.8	35.8934    	293.8  
10 	96    	279.56  	293.8	7.12       	293.8  
11 	97    	293.444 	293.8	2.492      	293.8  
12 	99    	293.8   	293.8	0          	293.8  
13 	94    	293.8   	293.8	0          	293.8  
14 	92    	293.8   	293.8	0          	293.8  
15 	98    	293.8   	293.8	0          	293.8  
16 	96    	293.8   	293.8	0          	293.8  
17 	93    	293.8   	293.8	0          	293.8  
18 	97    	290.472 	293.8	23.296     	293.8  
19 	95    	293.8   	293.8	0          	293.8  
20 	95    	29

[I 2023-10-15 19:42:07,424] Trial 1 finished with value: 324.60000000000076 and parameters: {'in_sigma': 0.9, 'in_tau': 0.8, 'in_mutpb': 1.0, 'pop_size': 50, 'n_offspring': 2, 'tournsize_parent': 8, 'cxpb': 0.9, 'cx_type': 'blend', 'alpha': 0.2, 'mutpb': 0.4, 'mut_type': 'adaptive', 'sigma': 1.0, 'indpb': 0.9, 'tau': 0.4, 'tournsize_survival': 6, 'ngen': 30, 'verbose': 1}. Best is trial 0 with value: 356.8000000000009.


...

30 	131   	258.256	334  	143.042	334    
gen	nevals	Mean   	Max  	Std    	max_hof
0  	50    	208.992	356.8	80.8319	356.8  
1  	100   	249.92 	356.8	161.048	356.8  
2  	100   	356.8  	356.8	5.68434e-14	356.8  
3  	100   	349.868	356.8	48.524     	356.8  
4  	100   	356.8  	356.8	5.68434e-14	356.8  
5  	100   	324.92 	356.8	109.264    	356.8  
6  	100   	349.7  	356.8	49.7       	356.8  
7  	100   	356.8  	356.8	5.68434e-14	356.8  
8  	100   	356.8  	356.8	5.68434e-14	356.8  
9  	100   	350.28 	356.8	45.64      	356.8  
10 	100   	356.8  	356.8	5.68434e-14	356.8  
11 	100   	349.344	356.8	52.192     	356.8  
12 	100   	356.8  	356.8	5.68434e-14	356.8  
13 	100   	356.8  	356.8	5.68434e-14	356.8  
14 	100   	350.54 	356.8	43.82      	356.8  
15 	100   	347.62 	356.8	64.26      	356.8  
16 	100   	347.764	356.8	63.252     	356.8  
17 	100   	339.104	356.8	87.1139    	356.8  
18 	100   	356.8  	356.8	5.68434e-14	356.8  
19 	100   	349.04 	356.8	54.32      	356.8  
20 	100   	356.8  	356.8	5

[I 2023-10-15 21:29:32,491] Trial 13 finished with value: 356.8000000000009 and parameters: {'in_sigma': 0.4, 'in_tau': 0.4, 'in_mutpb': 0.1, 'pop_size': 50, 'n_offspring': 2, 'tournsize_parent': 8, 'cxpb': 1.0, 'cx_type': 'whole_aritmetic', 'alpha': 0.8, 'mutpb': 0.5, 'mut_type': 'normal', 'sigma': 2.5, 'indpb': 0.7000000000000001, 'tau': 0.8, 'tournsize_survival': 6, 'ngen': 30, 'verbose': 1}. Best is trial 0 with value: 356.8000000000009.


30 	100   	356.8  	356.8	5.68434e-14	356.8  
gen	nevals	Mean  	Max  	Std    	max_hof
0  	50    	115.94	356.8	179.238	356.8  
1  	128   	219.1 	356.8	79.8206	356.8  
2  	124   	257.748	356.8	90.4555	356.8  
3  	116   	307.56 	356.8	26.9442	356.8  
4  	142   	318.3  	356.8	25.2042	356.8  
5  	131   	340.3  	356.8	25.2042	356.8  
6  	106   	356.8  	356.8	5.68434e-14	356.8  
7  	149   	356.8  	356.8	5.68434e-14	356.8  
8  	130   	356.8  	356.8	5.68434e-14	356.8  
9  	136   	356.8  	356.8	5.68434e-14	356.8  
10 	129   	356.8  	356.8	5.68434e-14	356.8  
11 	136   	356.8  	356.8	5.68434e-14	356.8  
12 	106   	356.8  	356.8	5.68434e-14	356.8  
13 	131   	356.8  	356.8	5.68434e-14	356.8  
14 	120   	356.8  	356.8	5.68434e-14	356.8  
15 	136   	356.8  	356.8	5.68434e-14	356.8  
16 	122   	356.8  	356.8	5.68434e-14	356.8  
17 	134   	356.8  	356.8	5.68434e-14	356.8  
18 	124   	356.8  	356.8	5.68434e-14	356.8  
19 	123   	356.8  	356.8	5.68434e-14	356.8  
20 	117   	356.8  	356.8	5.68434e-14	356.

[I 2023-10-15 21:40:24,561] Trial 14 finished with value: 356.8000000000009 and parameters: {'in_sigma': 0.30000000000000004, 'in_tau': 0.7000000000000001, 'in_mutpb': 0.30000000000000004, 'pop_size': 50, 'n_offspring': 5, 'tournsize_parent': 1, 'cxpb': 0.4, 'cx_type': 'whole_aritmetic', 'alpha': 0.30000000000000004, 'mutpb': 0.2, 'mut_type': 'adaptive', 'sigma': 2.0, 'indpb': 0.5, 'tau': 0.4, 'tournsize_survival': 5, 'ngen': 30, 'verbose': 1}. Best is trial 0 with value: 356.8000000000009.


30 	117   	356.8  	356.8	5.68434e-14	356.8  
gen	nevals	Mean   	Max  	Std    	max_hof
0  	50    	-24.716	356.8	341.051	356.8  
1  	48    	245.912	301.8	86.7559	356.8  
2  	47    	291.016	301.8	44.0278	356.8  
3  	49    	266.844	301.8	64.5635	356.8  
4  	47    	280.468	301.8	55.8628	356.8  
5  	47    	284.624	301.8	48.7203	356.8  
6  	49    	281.232	301.8	62.4932	356.8  
7  	48    	277.292	301.8	59.8721	356.8  
8  	46    	295.928	301.8	37.158 	356.8  
9  	47    	292.668	301.8	37.8336	356.8  
10 	50    	288.16 	301.8	46.3977	356.8  
11 	47    	275.784	301.8	63.5861	356.8  
12 	47    	288.768	301.8	44.2015	356.8  
13 	48    	281.84 	301.8	53.1571	356.8  
14 	46    	296.912	301.8	26.436 	356.8  
15 	49    	298.808	301.8	20.944 	356.8  
16 	45    	280.592	301.8	65.027 	356.8  


In [ ]:
#Trial 16 finished with value: 30.800000000000495 and parameters: {'initial_range': 2, 'pop_size': 50, 'n_offspring': 4, 'tournsize_parent': 6, 'cxpb': 0.6, 'cx_type': 'blend', 'alpha': 0.8, 'mutpb': 0.9, 'mut_type': 'adaptive', 'sigma': 3.0, 'indpb': 0.8, 'tau': 0.30000000000000004, 'tournsize_survival': 7, 'ngen': 30, 'verbose': 1}

In [ ]:
#Trial 0 finished with value: 356.8000000000009 and parameters: {'in_sigma': 0.7000000000000001, 'in_tau': 0.9, 'in_mutpb': 0.6, 'pop_size': 50, 'n_offspring': 4, 'tournsize_parent': 1, 'cxpb': 0.30000000000000004, 'cx_type': 'whole_aritmetic', 'alpha': 'random', 'mutpb': 0.9, 'mut_type': 'normal', 'sigma': 2.5, 'indpb': 0.1, 'tau': 0.8, 'tournsize_survival': 10, 'ngen': 30, 'verbose': 1}

In [ ]:
params = {'initial_range': 2,
          'pop_size': 50,
          'n_offspring': 4,
          'tournsize_parent': 6,
          'cxpb': 0.6,
          'cx_type': 'blend',
          'alpha': 0.8,
          'mutpb': 0.9,
          'mut_type': 'adaptive',
          'sigma': 3.0,
          'indpb': 0.8,
          'tau': 0.3,
          'tournsize_survival': 7,
          'ngen': 500,
          'verbose': 1}

In [ ]:
def final_res(sol):
    global env
    res = []
    for enemy in range(1,9):
        env.update_parameter("enemies",[enemy])
        res.append(env.play(sol))
    return res

In [ ]:
"""
params = {'initial_range':,
          'pop_size':,
          'n_offspring':,
          'tournsize_parent':,
          'cxpb':,
          'cx_type':,
          'alpha':,
          'mutpb':,
          'mut_type':,
          'sigma':,
          'indpb':,
          'tau':,
          'tournsize_survival':,
          'ngen':,
          'verbose':}
""""